In [11]:
import os
import sys
import pandas as pd
sys.path.append('../../')
from deepfold.data.utils.ontology import Ontology
from tools.evaluate_deepmodel import get_model_preds
from tools.evaluate_diamondscore import get_diamond_preds, get_diamond_scores

In [12]:
base_path = r'/home/niejianzheng/xbiome/datasets/protein'
# 存储所有数据文件路径
data_ls = os.walk(base_path).__next__()[2]
data_path_dict = {}
for data in data_ls:
    file_name = data.split('.')[0] + '_' + data.split('.')[1]
    data_path_dict[file_name] = os.path.join(base_path, data)

In [13]:
data_path_dict

{'test_data_fa': '/home/niejianzheng/xbiome/datasets/protein/test_data.fa',
 'uniprot_sprot_dat': '/home/niejianzheng/xbiome/datasets/protein/uniprot_sprot.dat',
 'test_diamond_res': '/home/niejianzheng/xbiome/datasets/protein/test_diamond.res',
 'test_data_pkl': '/home/niejianzheng/xbiome/datasets/protein/test_data.pkl',
 'go_obo': '/home/niejianzheng/xbiome/datasets/protein/go.obo',
 'train_data_pkl': '/home/niejianzheng/xbiome/datasets/protein/train_data.pkl',
 'esm1b_t33_650M_UR50S_embeddings_mean_train_pkl': '/home/niejianzheng/xbiome/datasets/protein/esm1b_t33_650M_UR50S_embeddings_mean_train.pkl',
 'train_data_dmnd': '/home/niejianzheng/xbiome/datasets/protein/train_data.dmnd',
 'predictions_pkl': '/home/niejianzheng/xbiome/datasets/protein/predictions.pkl',
 'esm1b_t33_650M_UR50S_embeddings_mean_test_pkl': '/home/niejianzheng/xbiome/datasets/protein/esm1b_t33_650M_UR50S_embeddings_mean_test.pkl',
 'terms_pkl': '/home/niejianzheng/xbiome/datasets/protein/terms.pkl',
 'swissprot_

## Diamond model

In [19]:
go_rels = Ontology(data_path_dict['go_obo'], with_rels=True)

train_df = pd.read_pickle(data_path_dict['train_data_pkl'])
annotations = train_df['prop_annotations'].values
annotations = list(map(lambda x: set(x), annotations))

test_df = pd.read_pickle(data_path_dict['test_data_pkl'])
test_annotations = test_df['prop_annotations'].values
test_annotations = list(map(lambda x: set(x), test_annotations))
go_rels.calculate_ic(annotations + test_annotations)

diamond_scores = get_diamond_scores(data_path_dict['test_diamond_res'])
blast_preds = get_diamond_preds(train_df, test_df, diamond_scores)

In [21]:
len(blast_preds)

3874

In [22]:
blast_preds[0]

{'GO:0000003': 0.11156404,
 'GO:0000131': 0.23063721,
 'GO:0000271': 0.10405492,
 'GO:0000280': 0.11156404,
 'GO:0003006': 0.11156404,
 'GO:0003674': 0.5462347,
 'GO:0003824': 0.5462347,
 'GO:0005488': 0.5462347,
 'GO:0005515': 0.5462347,
 'GO:0005575': 1.0,
 'GO:0005622': 0.45376527,
 'GO:0005628': 0.11156404,
 'GO:0005634': 0.34971038,
 'GO:0005737': 0.45376527,
 'GO:0005815': 0.23063721,
 'GO:0005816': 0.23063721,
 'GO:0005829': 0.45376527,
 'GO:0005856': 0.23063721,
 'GO:0005886': 0.20403346,
 'GO:0005933': 0.23063721,
 'GO:0005935': 0.23063721,
 'GO:0005975': 0.10405492,
 'GO:0005976': 0.10405492,
 'GO:0006022': 0.5462347,
 'GO:0006023': 0.5462347,
 'GO:0006030': 0.5462347,
 'GO:0006031': 0.5462347,
 'GO:0006037': 0.43467066,
 'GO:0006038': 0.43467066,
 'GO:0006040': 0.5462347,
 'GO:0006807': 0.5462347,
 'GO:0006996': 0.11156404,
 'GO:0007049': 0.11156404,
 'GO:0008047': 0.5462347,
 'GO:0008150': 0.65028965,
 'GO:0008152': 0.65028965,
 'GO:0009058': 0.65028965,
 'GO:0009059': 0.65

## DeeGoPlus Model